<a href="https://colab.research.google.com/github/SakshiHanamshetti/AI-LAB/blob/main/IMDBrecommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup



headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36',
           'Accept-Language': 'en-US, en;q=0.5'})


movie_name = []
year = []
time=[]
rating=[]
metascore =[]
director=[]
votes = []
gross = []
description = []
genre=[]
cast=[]
cas=[]
pages = np.arange(1,1000,50)
#https://www.imdb.com/search/title/?title_type=feature&primary_language=en
#https://www.imdb.com/search/title/?title_type=feature&primary_language=en&ref_=adv_prv
for page in pages:
   
    page = requests.get("https://www.imdb.com/search/title/?title_type=feature&primary_language=en&start="+str(page)+"&ref_=adv_nxt")
    soup = BeautifulSoup(page.text, 'html.parser')
    movie_data = soup.findAll('div', attrs = {'class': 'lister-item mode-advanced'})
    for store in movie_data:
        name = store.h3.a.text
        movie_name.append(name)
        
        year_of_release = store.h3.find('span', class_ = "lister-item-year text-muted unbold").text.replace('(', '')
        year_of_release=year_of_release.replace(')','')
        year.append(year_of_release)
        
        runtime = store.p.find("span", class_ = 'runtime').text if store.find('span', class_ = "runtime") else "NA"
        time.append(runtime)
        
        gen = store.p.find("span", class_ = 'genre').text
        genre.append(gen)
        
        rate = store.find('div', class_ = "inline-block ratings-imdb-rating").text.replace('\n', '') if store.find('div', class_ = "inline-block ratings-imdb-rating") else "NA"
        rating.append(rate)
        #rate = store.find('div', class_ = "ratings-bar").find('strong').text.replace('\n', '')
        #rating.append(rate)
        
        meta = store.find('span', class_ = "metascore").text if store.find('span', class_ = "metascore") else "NA"#if meta score not present then *
        
        metascore.append(meta)
        
        #dire=store.find('p',class_ = "metascore")
        dire=store.find('p',class_='').find_all('a')[0].text
        
        director.append(dire)
        
        #cas=([a.text for a in store.find('p',class_='').find_all('a')[1:]])
        #cast=','.join(map(str,cas))
        cast.append([a.text for a in store.find('p',class_='').find_all('a')[1:]])
      
        value = store.find_all('span', attrs = {'name':'nv'}) if store.find_all('span', attrs = {'name':'nv'}) else 'NA'
        vote = value[0].text if store.find_all('span', attrs = {'name':'nv'}) else 'NA'

        votes.append(vote)
        
        describe = store.find_all('p', class_ = 'text-muted')
        description_ = describe[1].text.replace('\n', '') if len(describe) >1 else 'NA'
        description.append(description_)
        
#dataframe
for i in cast:
    c=','.join(map(str,i))
    cas.append(c)
   
movie_list = pd.DataFrame({ "Movie Name": movie_name, "Year of Release" : year, "Watch Time": time,"Genre":genre,"Movie Rating": rating, "Metascore of movie": metascore,"Director":director,"Cast":cas,"Votes" : votes,"Description": description})
movie_list.to_excel("movie_data_new1.xlsx")

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pickle

In [18]:
data=pd.read_excel("movie_data_new1.xlsx")
data.rename(columns={'Unnamed: 0': 'movie_id'}, inplace=True)
columns=['Cast','Director','Genre','Movie Name','Description','Movie Rating']

In [27]:
def get_important_features(data):
    important_features=[]
    for i in range (0,data.shape[0]):
        important_features.append(str(data['Movie Name'][i]) + ' ' + str(data['Director'][i]) + ' ' + str(data['Genre'][i]) + ' ' + str(data['Description'][i]) + ' ' + str(data['Cast']) + ' ' + str(data['Movie Rating'][i]))
    return important_features

In [17]:
print(data.columns)


Index(['movie_id', 'Movie Name', 'Year of Release', 'Watch Time', 'Genre',
       'Movie Rating', 'Metascore of movie', 'Director', 'Cast', 'Votes',
       'Description'],
      dtype='object')


In [28]:
data['important_features'] = get_important_features(data)

In [29]:
tfidf = TfidfVectorizer(stop_words='english')
#data['Description'] = data['Description'].fillna('')
tfidf_matrix = tfidf.fit_transform(data['important_features'])
tfidf_matrix.shape

(1000, 6779)

In [30]:
print(data.shape)

(1000, 12)


In [32]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1.        , 0.0809269 , 0.10525309, ..., 0.08945696, 0.10167142,
        0.07471595],
       [0.0809269 , 1.        , 0.10968704, ..., 0.10898818, 0.15573178,
        0.09102876],
       [0.10525309, 0.10968704, 1.        , ..., 0.12124853, 0.13780382,
        0.10126881],
       ...,
       [0.08945696, 0.10898818, 0.12124853, ..., 1.        , 0.18886003,
        0.11664984],
       [0.10167142, 0.15573178, 0.13780382, ..., 0.18886003, 1.        ,
        0.12551199],
       [0.07471595, 0.09102876, 0.10126881, ..., 0.11664984, 0.12551199,
        1.        ]])

In [36]:
indices = pd.Series(data.index, index=data['Movie Name']).drop_duplicates()
#indices['Stillwater']
#sim_scores = list(enumerate(cosine_sim[indices['Stillwater']]))
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Sort the movies based on the similarity score
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    movie_indices = [i[0] for i in sim_scores]
    # Return the top 5 most similar movies
    movies=data['Movie Name'].iloc[movie_indices]
    id=data['movie_id'].iloc[movie_indices]
    dict={"Movies":movies,"id":id}
    final_df=pd.DataFrame(dict)
    final_df.reset_index(drop=True,inplace=True)
    return final_df

In [38]:
get_recommendations('Spider-Man: Far from Home')

,Movies,id
0,Spider-Man: No Way Home,58
1,Spider-Man: Beyond the Spider-Verse,31
2,Spider-Man: Homecoming,146
3,Spider-Man: Into the Spider-Verse,4
4,The Amazing Spider-Man 2,634


In [39]:
data.info()
new = data.drop(columns=['Year of Release','Watch Time','Genre','Movie Rating','Metascore of movie','Director','Cast','Votes','Description'])

pickle.dump(new,open('movie_list.pkl','wb'))
pickle.dump(cosine_sim,open('similarity.pkl','wb'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movie_id            1000 non-null   int64  
 1   Movie Name          1000 non-null   object 
 2   Year of Release     997 non-null    object 
 3   Watch Time          958 non-null    object 
 4   Genre               1000 non-null   object 
 5   Movie Rating        946 non-null    float64
 6   Metascore of movie  912 non-null    float64
 7   Director            1000 non-null   object 
 8   Cast                999 non-null    object 
 9   Votes               946 non-null    object 
 10  Description         1000 non-null   object 
 11  important_features  1000 non-null   object 
dtypes: float64(2), int64(1), object(9)
memory usage: 93.9+ KB


In [50]:
!pip install streamlit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [42]:
import pickle
import streamlit as st

def recommend(movie):
    index = movies[movies['Title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1])
    recommended_movie_names = []
    for i in distances[1:6]:
        recommended_movie_names.append(movies.iloc[i[0]].Title)

    return recommended_movie_names

page_bg_img = '''
<style>
      .stApp {
  background-image: url("https://payload.cargocollective.com/1/11/367710/13568488/MOVIECLASSICSerikweb_2500_800.jpg");
  background-size: cover;
}
</style>
'''

st.markdown(page_bg_img, unsafe_allow_html=True)


st.markdown('# Movie Recommendation System')
movies = pickle.load(open('movie_list.pkl','rb'))
similarity = pickle.load(open('similarity.pkl','rb'))

movie_list = movies['Movie Name'].values
selected_movie = st.selectbox(
    "Type or select a movie from the dropdown",
    movie_list
)


if st.button('Show Recommendation'):
    recommended_movie_names = recommend(selected_movie)
    for i in recommended_movie_names:
        st.subheader(i)

In [44]:
!python app.py

python3: can't open file '/content/app.py': [Errno 2] No such file or directory


In [45]:
import os

# Get the current working directory
current_directory = os.getcwd()

# List the files in the current directory
files_in_directory = os.listdir(current_directory)

# Print the current directory and the files in it
print("Current Directory:", current_directory)
print("Files in Directory:", files_in_directory)

Current Directory: /content
Files in Directory: ['.config', 'movie data_new1.xlsx', 'similarity.pkl', 'movie_data_new1.xlsx', 'movie_list.pkl', 'sample_data']


In [4]:
%%writefile app.py

import streamlit as st

def recommend(movie):
    index = movies[movies['Title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1])
    recommended_movie_names = []
    for i in distances[1:6]:
        recommended_movie_names.append(movies.iloc[i[0]].Title)

    return recommended_movie_names

page_bg_img = '''
<style>
      .stApp {
  background-image: url("https://payload.cargocollective.com/1/11/367710/13568488/MOVIECLASSICSerikweb_2500_800.jpg");
  background-size: cover;
}
</style>
'''

st.markdown(page_bg_img, unsafe_allow_html=True)


st.markdown('# Movie Recommendation System')
movies = pickle.load(open('movie_list.pkl','rb'))
similarity = pickle.load(open('similarity.pkl','rb'))

movie_list = movies['Movie Name'].values
selected_movie = st.selectbox(
    "Type or select a movie from the dropdown",
    movie_list
)


if st.button('Show Recommendation'):
    recommended_movie_names = recommend(selected_movie)
    for i in recommended_movie_names:
        st.subheader(i)


Overwriting app.py


In [ ]:
!streamlit run app.py




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.23.255.242:8501

